In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
from keras.optimizers import Adam
from scipy.spatial.distance import cdist

def load_and_preprocess_data():
    """Load and preprocess the MNIST dataset."""
    (x_train, _), (x_test, _) = mnist.load_data()
    x_train = np.expand_dims(x_train, axis=-1).astype('float32') / 255.
    x_test = np.expand_dims(x_test, axis=-1).astype('float32') / 255.
    return x_train, x_test

def build_autoencoder():
    """Build and compile the CNN-based autoencoder."""
    # Input layer: accepts images of shape 28x28x1 (MNIST images)
    input_img = Input(shape=(28, 28, 1))
    
    # Encoder
    # Convolutional layer with 32 filters, each 3x3, using 'relu' activation. 'same' padding ensures output size matches input size.
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    # Max pooling layer to reduce spatial dimensions by half, improving computational efficiency and helping encode positional information.
    x = MaxPooling2D((2, 2), padding='same')(x)
    # Another convolutional layer with 16 filters to further extract features from the image.
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    # Reducing spatial dimensions again to further compress the representation.
    x = MaxPooling2D((2, 2), padding='same')(x)
    # Final convolutional layer in the encoder with 8 filters, focusing on the most abstract features of the image.
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    # Last max pooling layer in the encoder to achieve the final compressed representation.
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    
    # Decoder
    # Convolutional layer with 8 filters, starting the process of decoding the compressed representation.
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    # Upsampling layer to start expanding the spatial dimensions back to the original size.
    x = UpSampling2D((2, 2))(x)
    # Convolutional layer with 16 filters to further refine the decoded features.
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    # Upsampling again to get closer to the original image size.
    x = UpSampling2D((2, 2))(x)
    # Convolutional layer with 32 filters, nearly restoring the original depth of features.
    x = Conv2D(32, (3, 3), activation='relu')(x)  # Note: No padding here, changes size slightly.
    # Final upsampling to match the original image dimensions.
    x = UpSampling2D((2, 2))(x)
    # Output layer to reconstruct the original image. Uses 'sigmoid' activation to output pixel values between 0 and 1.
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    # Compiling the autoencoder model with Adam optimizer and binary cross-entropy loss.
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
    return autoencoder


def train_autoencoder(autoencoder, x_train, x_test):
    """Train the autoencoder."""
    autoencoder.fit(x_train, x_train, epochs=1, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

def generate_embeddings(encoder, x_test):
    """Generate embeddings for the test set."""
    return encoder.predict(x_test)

def find_similar_images(artist_embeddings, copied_embeddings):
    """Find and return indices of similar images based on embeddings."""
    similar_images_indices = []
    for embedding in copied_embeddings:
        distances = cdist([embedding], artist_embeddings, 'euclidean')
        closest_indices = np.argsort(distances)[0][0:4]  # Exclude self
        similar_images_indices.append(closest_indices)
    return similar_images_indices

def display_similar_images(x_test, selected_indices, similar_images_indices):
    """Visualize the original and similar images."""
    plt.figure(figsize=(10, 7))
    for i, (index, sim_indices) in enumerate(zip(selected_indices, similar_images_indices)):
        ax = plt.subplot(3, 4, i * 4 + 1)
        plt.imshow(x_test[index].reshape(28, 28))
        plt.title("Original")
        plt.gray()
        ax.axis('off')
        
        for j, sim_index in enumerate(sim_indices):
            ax = plt.subplot(3, 4, i * 4 + j + 2)
            plt.imshow(x_test[sim_index].reshape(28, 28))
            plt.title(f"Similar {j+1}")
            plt.gray()
            ax.axis('off')
    plt.tight_layout()
    plt.show()

# Main workflow
x_train, x_test = load_and_preprocess_data()
autoencoder = build_autoencoder()
train_autoencoder(autoencoder, x_train, x_test)

encoder = Model(autoencoder.input, autoencoder.layers[-7].output)
encoded_imgs = generate_embeddings(encoder, np.reshape(x_test, (len(x_test), 28, 28, 1)))
encoded_imgs_flatten = encoded_imgs.reshape((len(x_test), np.prod(encoded_imgs.shape[1:])))

np.random.seed(0)
selected_indices = np.random.choice(x_test.shape[0], 3, replace=False)
print(encoded_imgs_flatten[selected_indices])
similar_images_indices = find_similar_images(encoded_imgs_flatten, encoded_imgs_flatten[selected_indices])
display_similar_images(x_test, selected_indices, similar_images_indices)

469/469 ━━━━━━━━━━━━━━━━━━━━ 74s 147ms/step - loss: 0.2961 - val_loss: 0.1314
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
[[1.6452286  0.         0.9446424  1.4481316  0.6801096  0.
  0.21253724 1.6036898  2.1334336  0.         1.1536818  1.9334687
  1.0276539  0.         1.1209863  0.12588558 1.1993984  0.
  1.1389915  2.1965058  0.9185519  0.         1.4154941  0.
  0.0939149  0.98624724 1.1080189  0.62442327 0.99399525 0.
  1.7361144  0.         1.6559418  0.         2.0530734  2.2301843
  0.46218324 0.07035834 0.         2.827939   0.25931057 0.
  1.1419768  3.2311444  0.4699456  0.         3.3032088  1.4368883
  0.         0.26433992 1.3918743  2.3706942  1.113142   0.
  3.6867752  0.         0.         1.1066552  1.4354508  1.2797693
  1.1876783  0.         2.3804173  0.         1.1407547  0.47485378
  1.8621093  1.6025746  0.8315718  0.         0.94038695 2.649459
  0.5292797  0.         1.6434708  3.4190927  0.83010584 0.21012408
  2.481738   1.408861   0.         0.         0.934

ValueError: XA must be a 2-dimensional array.